# Agentic RAG Using CrewAI & LangChain

We are going to see how agents can be involved in the RAG system to rtrieve the most relevant information.

Source: 

Plaban Nayak, Build A Local Reliable RAG Agent Using CrewAI And Groq, https://medium.com/the-ai-forum/build-a-local-reliable-rag-agent-using-crewai-and-groq-013e5d557bcd

Notebook:

https://github.com/pavanbelagatti/Agentic-RAG-LangChain-CrewAI/blob/main/crew-agentic-pav.ipynb

In [1]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq --quiet

In [2]:
import os
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from crewai_tools import PDFSearchTool

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.Skip

## Mention the Groq API Key

In [3]:
# Set the API key
GROQ_API_KEY = open("/home/mjack6/.secrets/groq_mjack.apikey", "r").read().strip()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

OPENAI_API_KEY = open("/home/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY 

## Mention the LLM being used

In [4]:
llm = ChatOpenAI(api_key=GROQ_API_KEY)

crew_llm = LLM(
    model="ollama/llama3-8b-8192",
    api_key=GROQ_API_KEY,
    max_tokens=1000,
    temperature=0.1
)

In [5]:
# Alternative Chat API:
# GEMINI_API_KEY = open("/Users/mjack6/.secrets/gemini_mjack.apikey", "r").read().strip()
# os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [6]:
# llm = ChatOpenAI(api_key=OPENAI_API_KEY)

# crew_llm = LLM(
#     model="gemini/gemini-1.5-flash",
#     api_key=GEMINI_API_KEY,
#     max_tokens=1000,
#     temperature=0.1
# )

In [7]:
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('NIPS-2017-attention-is-all-you-need-Paper.pdf', 'wb') as file:
    file.write(response.content)

## Create a RAG tool variable to pass our PDF

In [8]:
# rag_tool = PDFSearchTool(
#     pdf='NIPS-2017-attention-is-all-you-need-Paper.pdf',
#     config=dict(
#         llm=dict(
#             provider="groq", # or google, openai, anthropic, llama2, ...
#             config=dict(
#                 model="llama3-8b-8192",
#                 # temperature=0.5,
#                 # top_p=1,
#                 # stream=true,
#             ),
#         ),
#         embedder=dict(
#             provider="huggingface", # or openai, ollama, ...
#             config=dict(
#                 model="BAAI/bge-small-en-v1.5",
#                 # task_type="retrieval_document",
#                 # title="Embeddings",
#             ),
#         ),
#     )
# )

In [9]:
rag_tool = PDFSearchTool(
    pdf='NIPS-2017-attention-is-all-you-need-Paper.pdf',
    config=dict(
        llm=dict(
            provider="openai", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="openai", # or openai, ollama, ...
            config=dict(
                model="llama3-8b-8192",
                # task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


In [10]:
# result = rag_tool.run("How did self-attention mechanism evolve in large language models?")

# pprint(result)

## Mention the Tavily API Key

In [11]:
TAVILY_API_KEY = open("/home/mjack6/.secrets/tavily_mjack.apikey", "r").read().strip()
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [12]:
web_search_tool = TavilySearchResults(k=3)

In [13]:
web_search_tool.run("What is self-attention mechansim in large language models?")

[{'title': 'What is self-attention? | IBM',
  'url': 'https://www.ibm.com/think/topics/self-attention',
  'content': 'Self-attention is a type of attention mechanism used in machine learning models. This mechanism is used to\xa0weigh the importance of tokens or words in an input sequence to better understand the relations between them.\xa0It is a crucial part of transformer models, a powerful artificial intelligence architecture that is essential for natural language processing (NLP) tasks. The transformer architecture is the foundation for most modern large language models (LLMs). [...] NLP tasks:\xa0The self-attention mechanism enhances the linguistic capabilities of machine learning models by allowing the efficient and complete analysis of an entire text. Research has shown advancements in sentiment classification.6 Models can perform NLP tasks well because the attention layer allows it to compute the relation between words regardless of the distance between them.7 [...] Self-attent

## Define a Tool

In [14]:
@tool
def router_tool(question):
  """Router Function"""
  if 'self-attention' in question:
    return 'vectorstore'
  else:
    return 'web_search'

## Create Agents to work with

In [15]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [16]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [17]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [18]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [19]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

## Defining Tasks

In [20]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [21]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [22]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [23]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

## Define a flow for the use case

In [28]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

## Ask any query

In [31]:
#inputs = {"question":"How does self-attention mechanism help large language models?"}

inputs = {"question": "What is LORA as a technique with LLMs?"}

## Kick off the agent pipeline

In [32]:
result = rag_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c4bc33b7-a4df-4939-a921-19af284f956d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 087dd0a4-9974-433f-9b64-2d7204519ca5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 087dd0a4-9974-433f-9b64-2d7204519ca5
       Status: Executing Task...
    └── 🤖 Agent: Router
            Status: In Progress

# Agent: Router
## Task: Analyse the keywords in the question What is LORA as a technique with LLMs?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


🚀 Crew: crew
└── 📋 Task: 087dd0a4-9974-433f-9b64-2d7204519ca5
       Status: Executing Task...
    └── 🤖 Agent: Router
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Answer Grader
## Final Answer: 
Sorry! unable to find a valid response




🚀 Crew: crew
├── 📋 Task: 087dd0a4-9974-433f-9b64-2d7204519ca5
│      Assigned to: Router
│      Status: ✅ Completed
│   └── 🤖 Agent: Router
│           Status: ✅ Completed
│       └── 🔧 Failed router_tool (10)
├── 📋 Task: 9f5b3594-f03a-4ec1-9b04-194f8fec4fac
│      Assigned to: Retriever
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever
│           Status: ✅ Completed
├── 📋 Task: 52c3249c-ab88-4bdb-913d-cc0ea62fea17
│      Assigned to: Answer Grader
│      Status: ✅ Completed
│   └── 🤖 Agent: Answer Grader
│           Status: ✅ Completed
├── 📋 Task: 4dd1f2df-8c58-4cac-95ad-c7fec9fcfa03
│      Assigned to: Hallucination Grader
│      Status: ✅ Completed
│   └── 🤖 Agent: Hallucination Grader
│           Status: ✅ Completed
└── 📋 Task: 38c9625c-96f1-4a31-8cbd-bfef1c036ada
       Status: Executing Task...
    └── 🤖 Agent: Answer Grader
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 087dd0a4-9974-433f-9b64-2d7204519ca5
│      Assigned to: Router
│      Status: ✅ Completed
│   └── 🤖 Agent: Router
│           Status: ✅ Completed
│       └── 🔧 Failed router_tool (10)
├── 📋 Task: 9f5b3594-f03a-4ec1-9b04-194f8fec4fac
│      Assigned to: Retriever
│      Status: ✅ Completed
│   └── 🤖 Agent: Retriever
│           Status: ✅ Completed
├── 📋 Task: 52c3249c-ab88-4bdb-913d-cc0ea62fea17
│      Assigned to: Answer Grader
│      Status: ✅ Completed
│   └── 🤖 Agent: Answer Grader
│           Status: ✅ Completed
├── 📋 Task: 4dd1f2df-8c58-4cac-95ad-c7fec9fcfa03
│      Assigned to: Hallucination Grader
│      Status: ✅ Completed
│   └── 🤖 Agent: Hallucination Grader
│           Status: ✅ Completed
└── 📋 Task: 38c9625c-96f1-4a31-8cbd-bfef1c036ada
       Assigned to: Answer Grader
       Status: ✅ Completed
    └── 🤖 Agent: Answer Grader
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 38c9625c-96f1-4a31-8cbd-bfef1c036ada                                                                     │
│  Agent: Answer Grader                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c4bc33b7-a4df-4939-a921-19af284f956d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Obtain the final response

In [27]:
pprint(result.raw)

('The answer from the hallucination task is useful in resolving the question '
 'about how the self-attention mechanism helps large language models.')
